# YOLO26 Fish Detection - FiftyOne Integration

This notebook is designed to test YOLO26 using FiftyOne datasets.

## Capabilities:
- ✅ Load data from FiftyOne
- ✅ Run YOLO26 inference on test images
- ✅ Visualize results with bounding boxes
- ✅ Compare against FiftyOne ground truth
- ✅ Save predictions back to FiftyOne
- ✅ Interactive visualization in the FiftyOne App


## 1. Imports & Setup


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from ultralytics import YOLO
import fiftyone as fo
import fiftyone.zoo as foz
from fiftyone import ViewField as F
import warnings
warnings.filterwarnings('ignore')

# Configure matplotlib
%matplotlib inline
plt.rcParams['figure.figsize'] = (20, 12)
plt.rcParams['figure.dpi'] = 100

print("✅ Libraries imported")
print(f"FiftyOne version: {fo.__version__}")


## 2. Configuration


In [ ]:
# Label used for all ground truth and predictions
DETECTION_LABEL = "Fish"

#YOLO26 Medium 371 ms ± 1.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
#YOLO26 Nano  60.7 ms ± 1.05 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
#YOLO10 Nanon 173 ms ± 1.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
# ============== CONFIGURATION ==============

# FiftyOne dataset
FIFTYONE_DATASET_NAME = "segmentation_dataset_v0.10"

# Path to the trained model
MODEL_PATH = ""
# Detection parameters
CONFIDENCE_THRESHOLD = 0.25
IOU_THRESHOLD = 0.45
IMG_SIZE = 640

# Field name to store predictions in FiftyOne
PREDICTIONS_FIELD = "xyz"

# Field containing ground truth (Polylines segmentation)
GT_FIELD = 'General body shape'

# Visualization settings
BOX_THICKNESS = 3
FONT_SCALE = 1.0

print("✅ Configuration loaded")


## 3. Load FiftyOne Dataset


In [ ]:
# Load the dataset
print(f"Loading FiftyOne dataset: {FIFTYONE_DATASET_NAME}")
dataset = fo.load_dataset(FIFTYONE_DATASET_NAME)

print(f"\n✅ Dataset loaded")
print(f"Total samples: {len(dataset)}")
print(f"Media type: {dataset.media_type}")

# Check for available splits
print(f"\nAvailable tags: {dataset.count_values('tags')}")

# Retrieve the test split
test_view = dataset


## 4. Load YOLO26 Model


In [ ]:
print(f"Loading model: {MODEL_PATH}")
model = YOLO(MODEL_PATH)
# model.cpu()
print(f"\n✅ Model loaded successfully!")
print(f"Number of classes: {len(model.names)}")
print(f"Classes: {model.names}")


In [ ]:
# Take the first sample from the test view
sample = test_view.first()

print(f"Processing sample: {sample.id}")
print(f"File: {sample.filepath}")

# Run detection
%timeit model.predict(source=sample.filepath, conf=CONFIDENCE_THRESHOLD, iou=IOU_THRESHOLD, imgsz=IMG_SIZE, verbose=False)


## 5. Visualization Helper Functions


In [ ]:
def apply_mask_alpha(image, mask, color=(0, 255, 0), alpha=0.4):
    """
    Applies a binary mask to the image with alpha blending.
    The mask is automatically resized when the dimensions do not match.
    """
    output = image.copy()

    # Convert mask to uint8 and binarize
    mask = mask.astype(np.uint8)
    mask[mask > 0] = 1

    # Resize if dimensions do not match the image
    if mask.shape[:2] != image.shape[:2]:
        mask = cv2.resize(mask, (image.shape[1], image.shape[0]), interpolation=cv2.INTER_NEAREST)

    mask_bool = mask.astype(bool)

    colored_mask = np.zeros_like(image, dtype=np.uint8)
    colored_mask[mask_bool] = color

    output[mask_bool] = cv2.addWeighted(
        image[mask_bool],
        1 - alpha,
        colored_mask[mask_bool],
        alpha,
        0
    )

    return output


def draw_boxes_on_image(image, boxes, color=(0, 255, 0), thickness=BOX_THICKNESS, font_scale=FONT_SCALE, label_prefix=""):
    """Draws bounding boxes and labels for a list of detections.
    """
    output = image.copy()
    h, w = output.shape[:2]

    for detection in boxes:
        bbox = detection.get("bbox")
        if not bbox or len(bbox) != 4:
            continue

        x, y, box_w, box_h = bbox
        normalized = all(0 <= val <= 1 for val in (x, y, box_w, box_h))

        if normalized:
            x1 = int(round(x * w))
            y1 = int(round(y * h))
            x2 = int(round((x + box_w) * w))
            y2 = int(round((y + box_h) * h))
        else:
            x1 = int(round(x))
            y1 = int(round(y))
            x2 = int(round(x + box_w))
            y2 = int(round(y + box_h))

        x1, y1 = max(0, x1), max(0, y1)
        x2, y2 = min(w - 1, x2), min(h - 1, y2)

        cv2.rectangle(output, (x1, y1), (x2, y2), color, thickness)

        label = detection.get("label")
        confidence = detection.get("confidence")

        text_parts = []
        if label:
            text_parts.append(f"{label_prefix}{label}")
        elif label_prefix:
            text_parts.append(label_prefix.strip())

        if confidence is not None:
            text_parts.append(f"{confidence:.2f}")

        if text_parts:
            text = " ".join(text_parts)
            text_size, baseline = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, font_scale, 1)
            text_width, text_height = text_size
            text_origin = (x1, max(0, y1 - text_height - baseline - 4))
            text_bg_top_left = (text_origin[0], text_origin[1])
            text_bg_bottom_right = (
                text_origin[0] + text_width + 4,
                text_origin[1] + text_height + baseline + 4,
            )

            cv2.rectangle(output, text_bg_top_left, text_bg_bottom_right, color, cv2.FILLED)
            cv2.putText(
                output,
                text,
                (text_origin[0] + 2, text_origin[1] + text_height + baseline),
                cv2.FONT_HERSHEY_SIMPLEX,
                font_scale,
                (255, 255, 255),
                1,
                cv2.LINE_AA,
            )

    return output


def visualize_sample_with_predictions(sample, predictions, show_ground_truth=True):
    """
    Visualizes a sample with YOLO predictions (boxes + masks) and optional ground truth.
    """
    image = cv2.imread(sample.filepath)
    if image is None:
        print(f"⚠️ Failed to load image: {sample.filepath}")
        return

    h, w = image.shape[:2]
    result = predictions[0]

    # ----------------------------
    # YOLO predictions
    # ----------------------------
    pred_boxes = []
    pred_masks = []

    has_masks = result.masks is not None
    img_preds = image.copy()
    
    for i, box in enumerate(result.boxes):
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0])

        if conf < CONFIDENCE_THRESHOLD:
            continue

        cls = int(box.cls[0])

        pred_boxes.append({
            'bbox': [x1 / w, y1 / h, (x2 - x1) / w, (y2 - y1) / h],
            'label': model.names[cls],
            'confidence': conf
        })

        if has_masks and hasattr(result.masks, "xy"):
            pred_masks.append(result.masks.data[i].cpu().numpy())
            for polygon in result.masks.xy:  # list of polygons
                # polygon.shape = [num_points, 2]
                pts = np.array(polygon, np.int32)
                pts = pts.reshape((-1, 1, 2))  # needed for cv2.polylines
                cv2.polylines(img_preds, [pts], isClosed=True, color=POLYGON_COLOR, thickness=6)


    # ----------------------------
    # Ground truth
    # ----------------------------
    gt_boxes = []
    if show_ground_truth and GT_FIELD in sample.field_names and sample[GT_FIELD]:
        for polyline in sample[GT_FIELD].polylines:
            all_points = [p for segment in polyline.points for p in segment]
            if all_points:
                xs = [p[0] for p in all_points]
                ys = [p[1] for p in all_points]
                x_min, x_max = min(xs), max(xs)
                y_min, y_max = min(ys), max(ys)
                gt_boxes.append({
                    'bbox': [x_min, y_min, x_max - x_min, y_max - y_min],
                    'label': polyline.label if hasattr(polyline, 'label') else 'fish',
                })

    # ----------------------------
    # Visualization
    # ----------------------------
    num_plots = 3 if show_ground_truth and gt_boxes else 2
    fig, axes = plt.subplots(1, num_plots, figsize=(20, 8))
    if num_plots == 1:
        axes = [axes]

    # Original
    axes[0].imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    axes[0].set_title("Original", fontsize=16, fontweight="bold")
    axes[0].axis("off")

    # Predictions (masks -> boxes)
    

    for mask in pred_masks:
        img_preds = apply_mask_alpha(
            img_preds,
            mask,
            color=MASK_COLOR,
            alpha=MASK_ALPHA
        )

    img_preds = draw_boxes_on_image(
        img_preds,
        pred_boxes,
        color=MASK_COLOR,
        label_prefix=""
    )

    axes[1].imshow(cv2.cvtColor(img_preds, cv2.COLOR_BGR2RGB))
    axes[1].set_title(
        f"YOLO Predictions ({len(pred_boxes)})",
        fontsize=16,
        fontweight="bold",
        color="green"
    )
    axes[1].axis("off")

    # Ground truth
    if show_ground_truth and gt_boxes:
        img_gt = draw_boxes_on_image(
            image,
            gt_boxes,
            color=(0, 0, 255),
            label_prefix="GT: "
        )
        axes[2].imshow(cv2.cvtColor(img_gt, cv2.COLOR_BGR2RGB))
        axes[2].set_title(
            f"Ground Truth ({len(gt_boxes)})",
            fontsize=16,
            fontweight="bold",
            color="red"
        )
        axes[2].axis("off")

    plt.tight_layout()
    plt.show()

    # ----------------------------
    # Stats
    # ----------------------------
    print(f"\n{'=' * 70}")
    print(f"Sample ID: {sample.id}")
    print(f"File: {Path(sample.filepath).name}")
    print(f"Size: {w}x{h}")
    print(f"Predicted: {len(pred_boxes)} objects")
    if gt_boxes:
        print(f"Ground truth: {len(gt_boxes)} objects")
    print(f"{'=' * 70}")


## 6. Single Image Test


## 7. Multiple Image Test


In [ ]:

MASK_COLOR = (0, 255, 0)   # BGR
POLYGON_COLOR = (255, 255, 0)   # BGR
MASK_ALPHA = 0.4

# Take a few random samples
NUM_SAMPLES = 1000
samples = test_view.take(NUM_SAMPLES)

print(f"Processing {len(samples)} images\n")

for sample in samples:
    if len(sample['General body shape']['polylines']) > 1:
        # Run detection
        results = model.predict(
            source=sample.filepath,
            conf=CONFIDENCE_THRESHOLD,
            iou=IOU_THRESHOLD,
            imgsz=IMG_SIZE,
            verbose=False
        )

        # Visualization
        visualize_sample_with_predictions(sample, results, show_ground_truth=True)
        print("\n" + "="*80 + "\n")


## 8. Run Detection on the Full Test Split


In [ ]:
import fiftyone as fo
import cv2
import numpy as np
from tqdm import tqdm
PREDICTIONS_FIELD = ''

print(f"Running detection on {len(test_view)} images...")
print(f"Results will be stored in field: '{PREDICTIONS_FIELD}'\n")

for i, sample in tqdm(enumerate(test_view), total=len(test_view)):
    
    # Run YOLO detection
    results = model.predict(
        source=sample.filepath,
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        imgsz=IMG_SIZE,
        verbose=False
    )
    
    detections = []
    polygons = []
    img = cv2.imread(sample.filepath)
    h, w = img.shape[:2]
    
    result = results[0]
    
    # ----------------------------
    # Process bounding boxes
    # ----------------------------
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0])
        if conf < CONFIDENCE_THRESHOLD:
            continue
        cls = int(box.cls[0])
        
        det = fo.Detection(
            label=DETECTION_LABEL,
            bounding_box=[x1/w, y1/h, (x2-x1)/w, (y2-y1)/h],
            confidence=conf
        )
        detections.append(det)
    
    # ----------------------------
    # Process polygons if available
    # ----------------------------
    if hasattr(result.masks, "xy") and result.masks.xy is not None:
        for poly in result.masks.xy:
            # poly.shape = [num_points, 2]
            pts = [[float(x)/w, float(y)/h] for x, y in poly]  # normalize
            polygons.append(fo.Polyline(points=[pts], label=DETECTION_LABEL))  # Keep [pts] wrapper
    
    # ----------------------------
    # Save everything to the sample
    # ----------------------------
    # Primary detections
    sample[PREDICTIONS_FIELD] = fo.Detections(detections=detections)
    
    # Additionally store polygons in a separate field, if present
    if polygons:
        sample[f"{PREDICTIONS_FIELD}_polygons"] = fo.Polylines(polylines=polygons)
    
    sample.save()

print(f"\n✅ Detection finished! Predictions saved to field '{PREDICTIONS_FIELD}'")


In [ ]:
import fiftyone as fo
import cv2
import numpy as np
from tqdm import tqdm
PREDICTIONS_FIELD = 'SIMPLE_12'

print(f"Running detection on {len(test_view)} images...")
print(f"Results will be stored in field: '{PREDICTIONS_FIELD}'\n")

for i, sample in tqdm(enumerate(test_view), total=len(test_view)):
    # if i % 10 == 0:
    #     print(f"Processed: {i}/{len(test_view)}")
    
    # Run YOLO detection
    results = model.predict(
        source=sample.filepath,
        conf=CONFIDENCE_THRESHOLD,
        iou=IOU_THRESHOLD,
        imgsz=IMG_SIZE,
        verbose=False
    )
    
    detections = []
    polygons = []
    img = cv2.imread(sample.filepath)
    h, w = img.shape[:2]
    
    result = results[0]
    
    # ----------------------------
    # Process bounding boxes
    # ----------------------------
    for box in result.boxes:
        x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
        conf = float(box.conf[0])
        if conf < CONFIDENCE_THRESHOLD:
            continue
        cls = int(box.cls[0])
        
        det = fo.Detection(
            label=DETECTION_LABEL,
            bounding_box=[x1/w, y1/h, (x2-x1)/w, (y2-y1)/h],
            confidence=conf
        )
        detections.append(det)
    
    # ----------------------------
    # Process polygons if available
    # ----------------------------
    if hasattr(result.masks, "xy") and result.masks.xy is not None:
        for poly in result.masks.xy:
            # poly.shape = [num_points, 2]
            pts = [[float(x)/w, float(y)/h] for x, y in poly]  # normalize
            polygons.append(fo.Polyline(points=[pts], label=DETECTION_LABEL))  # Keep [pts] wrapper
    
    # ----------------------------
    # Save everything to the sample
    # ----------------------------
    # Primary detections
    sample[PREDICTIONS_FIELD] = fo.Detections(detections=detections)
    
    # Additionally store polygons in a separate field, if present
    if polygons:
        sample[f"{PREDICTIONS_FIELD}_polygons"] = fo.Polylines(polylines=polygons)
    
    sample.save()

print(f"\n✅ Detection finished! Predictions saved to field '{PREDICTIONS_FIELD}'")


## 9. Prediction Statistics


In [ ]:
import pandas as pd
from tqdm import tqdm

# Collect statistics
stats = []

for sample in tqdm(dataset):
    # Number of predictions
    preds = sample[PREDICTIONS_FIELD]
    num_preds = len(preds.detections) if preds else 0
    
    # Number of ground truth objects (from Polylines in the GT_FIELD)
    gt = sample[GT_FIELD] if GT_FIELD in sample.field_names else None
    num_gt = len(gt.polylines) if gt else 0
    
    # Average confidence
    if preds and preds.detections:
        confidences = [d.confidence for d in preds.detections]
        avg_conf = np.mean(confidences)
        max_conf = max(confidences)
        min_conf = min(confidences)
    else:
        avg_conf = max_conf = min_conf = 0
    
    stats.append({
        'Sample ID': sample.id[:8] + '...',
        'Filename': Path(sample.filepath).name,
        'Predictions': num_preds,
        'Ground Truth': num_gt,
        'Avg Conf': f"{avg_conf:.3f}",
        'Max Conf': f"{max_conf:.3f}",
        'Min Conf': f"{min_conf:.3f}"
    })

# Create DataFrame
df = pd.DataFrame(stats)

print("\n" + "="*100)
print("📊 DETECTION STATISTICS")
print("="*100)
print(df.head(20).to_string(index=False))
print("\n... (showing first 20 rows)\n")

# Overall statistics
total_preds = df['Predictions'].sum()
total_gt = df['Ground Truth'].sum()
avg_preds_per_img = df['Predictions'].mean()
imgs_with_preds = len(df[df['Predictions'] > 0])

print("="*100)
print("📈 OVERALL STATISTICS")
print("="*100)
print(f"Total images:                    {len(test_view)}")
print(f"Images with predictions:         {imgs_with_preds} ({imgs_with_preds/len(test_view)*100:.1f}%)")
print(f"Total predictions:               {total_preds}")
print(f"Total ground truth:              {total_gt}")
print(f"Average predictions per image:  {avg_preds_per_img:.2f}")
print(f"Average GT per image:            {total_gt/len(test_view):.2f}")
print("="*100)


## 10. Evaluation and Validation

We compute COCO-style metrics for the YOLO26 predictions using IoU thresholds 0.5 and 0.95, and store the results in the dataset for further analysis.


In [ ]:
GT_DETECTIONS_FIELD = "gt_bodyshape_detections"
EVAL_KEY = ''
COCO_IOU_THRESHOLDS = [0.5, 0.95]
print(f"EVAL_KEY: {EVAL_KEY}")

def _polylines_to_detections(polyline_field):
    detections = []
    if not polyline_field:
        return detections

    for polyline in polyline_field.polylines:
        points = [pt for segment in polyline.points for pt in segment]
        if not points:
            continue

        xs = [pt[0] for pt in points]
        ys = [pt[1] for pt in points]
        x_min, x_max = min(xs), max(xs)
        y_min, y_max = min(ys), max(ys)

        width = max(1e-6, min(1.0, x_max - x_min))
        height = max(1e-6, min(1.0, y_max - y_min))
        x_min = max(0.0, min(1.0, x_min))
        y_min = max(0.0, min(1.0, y_min))

        label = DETECTION_LABEL
        detections.append(
            fo.Detection(
                label="Fish",
                bounding_box=[x_min, y_min, width, height],
            )
        )

    return detections


if len(test_view) == 0:
    print("⚠️ No test images available for evaluation")
else:
    first_sample = dataset.first()
    if first_sample is not None and GT_DETECTIONS_FIELD not in first_sample.field_names:
        print("Creating a field with detected GT bboxes...")
        for sample in test_view:
            gt_field = sample[GT_FIELD] if GT_FIELD in sample.field_names else None
            sample[GT_DETECTIONS_FIELD] = fo.Detections(
                detections=_polylines_to_detections(gt_field)
            )
            sample.save()
        print(f"✅ Field '{GT_DETECTIONS_FIELD}' added for {len(test_view)} samples")
    elif first_sample is not None:
        print(f"Field '{GT_DETECTIONS_FIELD}' already exists, skipping transformation")

    print("\nRunning COCO evaluation (compute_mAP + IoU sweep)")
    coco_results = test_view.evaluate_detections(
        PREDICTIONS_FIELD,
        gt_field=GT_DETECTIONS_FIELD,
        eval_key="",
        method="coco",
        iou=0.5,
        classwise=False,
        compute_mAP=True,
        iou_threshs=COCO_IOU_THRESHOLDS,
        max_preds=500,
    )

    def _mean_positive(values):
        arr = np.array(values, dtype=float)
        valid = arr[arr > -1]
        if valid.size == 0:
            return float("nan")
        return float(valid.mean())

    def _ap_at(iou_thresh):
        idx = coco_results._get_iou_thresh_inds(iou_thresh)[0]
        per_class_ap = np.mean(coco_results.precision[idx], axis=1)
        return _mean_positive(per_class_ap)

    def _ar_at(iou_thresh):
        if coco_results.recall_sweep is None:
            return float("nan")
        idx = coco_results._get_iou_thresh_inds(iou_thresh)[0]
        per_class_ar = np.array(coco_results.recall_sweep[idx], dtype=float)
        return _mean_positive(per_class_ar)

    tp_total = sum(value or 0 for value in test_view.values(f"{EVAL_KEY}_tp"))
    fp_total = sum(value or 0 for value in test_view.values(f"{EVAL_KEY}_fp"))
    fn_total = sum(value or 0 for value in test_view.values(f"{EVAL_KEY}_fn"))

    classification_metrics = coco_results.metrics()

    print("\n" + "=" * 90)
    print("🧮 COCO-style metrics")
    print("=" * 90)
    print(f"mAP (IoU 0.5:0.95):          {coco_results.mAP():.3f}")
    print(f"mAR (IoU 0.5:0.95):          {coco_results.mAR():.3f}")
    print(f"AP @ 0.5:                    {_ap_at(0.5):.3f}")
    print(f"AP @ 0.95:                   {_ap_at(0.95):.3f}")
    print(f"AR @ 0.5:                    {_ar_at(0.5):.3f}")
    print(f"AR @ 0.95:                   {_ar_at(0.95):.3f}")

    print("\n" + "=" * 90)
    print("⚖️ Classification metrics")
    print("=" * 90)
    print(f"Precision (micro):            {classification_metrics['precision']:.3f}")
    print(f"Recall (micro):               {classification_metrics['recall']:.3f}")
    print(f"F1-score (micro):             {classification_metrics['fscore']:.3f}")
    print(f"Accuracy:                     {classification_metrics['accuracy']:.3f}")
    print(f"Support:                      {classification_metrics['support']}")

    print("\n" + "=" * 90)
    print("🔢 TP/FP/FN counters / objects")
    print("=" * 90)
    print(f"TP: {tp_total}, FP: {fp_total}, FN: {fn_total}")
    print(f"Total GT objects:            {total_gt}")
    print(f"Total predictions:           {total_preds}")
    print("=" * 90)


In [ ]:
GT_DETECTIONS_FIELD = "sam3_segmentation_rect"
EVAL_KEY = ''
COCO_IOU_THRESHOLDS = [0.5, 0.95]
print(f"EVAL_KEY: {EVAL_KEY}")
GT_FIELD = 'sam3_segmentation'

print("Creating a field with detected GT bboxes...")
for sample in tqdm(test_view):
    gt_field = sample[GT_FIELD] if GT_FIELD in sample.field_names else None
    sample[GT_DETECTIONS_FIELD] = fo.Detections(
        detections=_polylines_to_detections(gt_field)
    )
    sample.save()
print(f"✅ Field '{GT_DETECTIONS_FIELD}' added for {len(test_view)} samples")


## 11. FiftyOne App Visualization


In [ ]:
PREDICTIONS_FIELD = ''
GT_DETECTIONS_FIELD = 'sam3_segmentation_rect'
print("\nRunning COCO evaluation (compute_mAP + IoU sweep)")
coco_results = test_view.evaluate_detections(
    PREDICTIONS_FIELD,
    gt_field=GT_DETECTIONS_FIELD,
    eval_key=None,
    method="coco",
    iou=0.5,
    classwise=True,
    compute_mAP=True,
    iou_threshs=np.arange(0.5, 0.96, 0.05),
    max_preds=500,
)

print("\n" + "=" * 90)
print("🧮 COCO-style metrics")
print("=" * 90)
print(f"mAP (IoU 0.5:0.95):          {coco_results.mAP():.3f}")
print(f"mAR (IoU 0.5:0.95):          {coco_results.mAR():.3f}")
print(f"AP @ 0.5:                    {_ap_at(0.5):.3f}")
print(f"AP @ 0.95:                   {_ap_at(0.95):.3f}")
print(f"AR @ 0.5:                    {_ar_at(0.5):.3f}")
print(f"AR @ 0.95:                   {_ar_at(0.95):.3f}")


In [ ]:
conf_thresholds = np.arange(0.1, 1.0, 0.05)
best_mAP = 0
best_thr = 0
for thr in conf_thresholds:
    coco_results = test_view.evaluate_detections(
        "",
        gt_field="sam3_segmentation_rect",
        method="coco",
        iou=0.5,
        classwise=True,
        compute_mAP=True,
        max_preds=500,
        conf_threshold=thr  # <-- set the threshold here
    )
    mAP = coco_results.mAP()
    print(f"Current conf threshold: {best_thr}, mAP={best_mAP:.3f}")
    if mAP > best_mAP:
        best_mAP = mAP
        best_thr = thr
        print(f"Best conf threshold: {best_thr}, mAP={best_mAP:.3f}")

print(f"Best conf threshold: {best_thr}, mAP={best_mAP:.3f}")


In [ ]:
coco_results.print_report(classes=['Fish'])

V09
              precision    recall  f1-score   support

        Fish       0.98      0.70      0.81     83223

   micro avg       0.98      0.70      0.81     83223
   macro avg       0.98      0.70      0.81     83223
weighted avg       0.98      0.70      0.81     83223

fish_detection_20260203_130548

              precision    recall  f1-score   support

        Fish       0.94      0.79      0.86     83223

   micro avg       0.94      0.79      0.86     83223
   macro avg       0.94      0.79      0.86     83223
weighted avg       0.94      0.79      0.86     83223

yolo26_v10_run_5_best
            precision    recall  f1-score   support

        Fish       0.97      0.69      0.80     83223

   micro avg       0.97      0.69      0.80     83223
   macro avg       0.97      0.69      0.80     83223
weighted avg       0.97      0.69      0.80     83223

fish_detection_20260204_115831
              precision    recall  f1-score   support

        Fish       0.94      0.80      0.86     83223

   micro avg       0.94      0.80      0.86     83223
   macro avg       0.94      0.80      0.86     83223
weighted avg       0.94      0.80      0.86     83223

fish_detection_20260205_211724
              precision    recall  f1-score   support

        Fish       0.94      0.79      0.86     83223

   micro avg       0.94      0.79      0.86     83223
   macro avg       0.94      0.79      0.86     83223
weighted avg       0.94      0.79      0.86     83223

SAMPLE 12
                 precision    recall  f1-score   support

        Fish       0.94      0.78      0.85     83223

   micro avg       0.94      0.78      0.85     83223
   macro avg       0.94      0.78      0.85     83223
weighted avg       0.94      0.78      0.85     83223


SAMPLE MEDIUM 2 Update 16/02

              precision    recall  f1-score   support

        Fish       0.94      0.86      0.90     83223

   micro avg       0.94      0.86      0.90     83223
   macro avg       0.94      0.86      0.90     83223
weighted avg       0.94      0.86      0.90     83223

SAMPLE_MEDIUM2 TEST Dataset

              precision    recall  f1-score   support

        Fish       0.89      1.00      0.94     20849

   micro avg       0.89      1.00      0.94     20849
   macro avg       0.89      1.00      0.94     20849
weighted avg       0.89      1.00      0.94     20849
